In [1]:
import pandas as pd
import statsmodels.api as sm


In [2]:
# Load data
df = pd.read_csv("data/ESS11.csv", sep=";", encoding="utf-8", 
                usecols=["idno", "respc19a", "vacc19",	"cntry",
                        "eisced","hinctnta", "hhmmb", "netusoft", 
                        "gndr", "maritalb", "domicil", "agea", "trstprl",
                        "trstlgl", "trstplc", "trstplt", "trstprt"])

In [3]:
# Show basic structure
print("Shape of the data:", df.shape)

Shape of the data: (46162, 17)


In [4]:
# Display first five rows
df.head()

,idno,respc19a,vacc19,cntry,eisced,hinctnta,hhmmb,netusoft,gndr,maritalb,domicil,agea,trstprl,trstlgl,trstplc,trstplt,trstprt
0,50014,1.0,1.0,AT,3.0,6.0,2.0,5.0,1,1.0,3.0,65.0,6.0,9.0,10.0,5.0,5.0
1,50030,1.0,1.0,AT,5.0,1.0,1.0,5.0,2,6.0,1.0,21.0,6.0,6.0,4.0,1.0,0.0
2,50057,1.0,1.0,AT,6.0,5.0,3.0,5.0,2,1.0,3.0,53.0,7.0,5.0,8.0,4.0,4.0
3,50106,1.0,1.0,AT,5.0,2.0,1.0,1.0,2,4.0,1.0,78.0,5.0,6.0,9.0,3.0,3.0
4,50145,3.0,1.0,AT,3.0,NaN,2.0,5.0,1,1.0,4.0,64.0,6.0,8.0,8.0,5.0,5.0


Dependent variable encoding

The variable vacc19 tells us whether someone has received at least one COVID-19 vaccination dose. People could answer:

1 = Yes

2 = No

7 = Refusal to answer

8 = Don’t know

9 = No answer

Since we want to model who actually got vaccinated versus who did not, it makes sense to treat:

1 → vaccinated (coded as 1)

2 → not vaccinated (coded as 0)

The other responses (7, 8, 9) do not give us any meaningful information. They just mean the person refused, didn’t know, or skipped the question. Therefore, we should treat them as missing values and remove those rows from the analysis.

In simple words, if people refused to answer, we cannot know their vaccination status — so we have to drop those rows.

In [5]:
# Recode vacc19
df = df[df['vacc19'].isin([1, 2])]  # keep only valid responses
df['vacc19_binary'] = df['vacc19'].map({1: 1, 2: 0})

Research Question:
Which demographic, socioeconomic, and attitudinal factors influenced COVID-19 vaccination uptake in Germany in 2023?

In [6]:
# Recode respc19a as binary (1 = ever infected, 0 = never infected)
df = df[df['respc19a'].isin([1, 2, 3])]
df['respc19a_binary'] = df['respc19a'].map({1: 1, 2: 1, 3: 0})

In [7]:
# Drop missing categories for other variables according to codebook
categorical_vars = ['gndr', 'maritalb', 'domicil', 'cntry', 'eisced']
for var in categorical_vars:
    df = df[~df[var].isin([7,8,9,77,88,99])]

In [8]:
# Filter out impossible/missing age
df = df[df['agea'] != 999]

# (optional) if you want to limit to one country, e.g. Germany:
df = df[df['cntry'] == 'DE']

Question 4: linear combination check

In [9]:
ctab = None

In [10]:
# check cross-tabulations for each categorical variable before fitting:
for var in ['gndr', 'maritalb', 'domicil', 'cntry', 'eisced', "respc19a_binary"]:
    ctab = pd.crosstab(df[var], df['vacc19_binary'])
    print(f"\nCross-tabulation for {var}:\n", ctab)



Cross-tabulation for gndr:
 vacc19_binary   0    1
gndr                  
1              67  918
2              98  929

Cross-tabulation for maritalb:
 vacc19_binary   0    1
maritalb              
1.0            74  962
2.0             2    8
4.0             9  168
5.0             4  129
6.0            74  573

Cross-tabulation for domicil:
 vacc19_binary   0    1
domicil               
1.0            18  297
2.0            16  236
3.0            58  597
4.0            66  677
5.0             7   38

Cross-tabulation for cntry:
 vacc19_binary    0     1
cntry                   
DE             165  1847

Cross-tabulation for eisced:
 vacc19_binary   0    1
eisced                
1.0            10   32
2.0            33  151
3.0            67  828
4.0            12  104
5.0            30  518
6.0            12  203

Cross-tabulation for respc19a_binary:
 vacc19_binary      0     1
respc19a_binary           
0                 55   607
1                110  1240


In [11]:
# Assuming df is still loaded from previous cleaning steps

# Filter for Germany
df = df[df['cntry'] == 'DE']

# Redefine the list of predictors
predictors = [
    'agea',
    'respc19a_binary',
    'hinctnta',
    'hhmmb',
    'netusoft',
    'trstprl',
    'trstlgl',
    'trstplc',
    'trstplt',
    'trstprt'
]

In [12]:
# Dummy code relevant categorical variables
df = pd.get_dummies(df, columns=['gndr', 'maritalb', 'domicil', 'eisced'], drop_first=True)

# Add all dummies to predictors
predictors += [col for col in df.columns if any(x in col for x in ['gndr_', 'maritalb_', 'domicil_', 'eisced_'])]

# Make sure there are no non-numeric columns:
for col in predictors:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Remove any rows with NA (if missing)
df_model = df.dropna(subset=predictors + ['vacc19_binary'])

In [13]:
# Create design matrix
X = df_model[predictors]
X = sm.add_constant(X)

# Response variable
y = df_model['vacc19_binary']

# Confirm again that X is fully numeric
print(X.dtypes)

# Fit the model
logit_model = sm.Logit(y, X)
result = logit_model.fit()


const              float64
agea               float64
respc19a_binary      int64
hinctnta           float64
hhmmb              float64
netusoft           float64
trstprl            float64
trstlgl            float64
trstplc            float64
trstplt            float64
trstprt            float64
gndr_2                bool
maritalb_2.0          bool
maritalb_4.0          bool
maritalb_5.0          bool
maritalb_6.0          bool
domicil_2.0           bool
domicil_3.0           bool
domicil_4.0           bool
domicil_5.0           bool
eisced_2.0            bool
eisced_3.0            bool
eisced_4.0            bool
eisced_5.0            bool
eisced_6.0            bool
dtype: object


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [23]:
# Fit the logistic regression
logit_model = sm.Logit(y, X)
result = logit_model.fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
# Print the summary
print(result.summary())

In [ ]:
# You can also get odds ratios:
odds_ratios = pd.Series(np.exp(result.params), name="Odds Ratio")
print("\nOdds Ratios:\n", odds_ratios)